In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### Подключение библиотек и импорт данных

In [ ]:
%matplotlib inline

import datetime
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import re
import nltk

from folium import plugins
!pip install alphashape
import alphashape
from functools import lru_cache
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from collections import Counter


cols = ['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude ', 'missed']
df1 = pd.read_csv('/kaggle/input/ufo-sightings/complete.csv', error_bad_lines=False, names=cols)
df1.drop([0], inplace=True)
df2 = pd.read_csv('/kaggle/input/ufo-sightings/scrubbed.csv')
df2.head()

In [ ]:
print(f'Shape of complete.csv:', df1.shape)
print(f'Shape of scrubbed.csv:', df2.shape)

In [ ]:

def na_cols_freq(df):
    features_num = df.columns
    na_columns = df.isna().sum(axis=0)
    na_columns_num = na_columns[features_num] / df.shape[0]
    print(na_columns_num[na_columns_num > 0], '\n')

uncorrect = df1.loc[df1['missed'] == 0]
print('Uncorrect rows:', uncorrect.shape[0], '\n')
print('Nan elements missed:\n')
na_cols_freq(uncorrect)
uncorrect.head()

Количество неправильно считанных данных (из-за наличия дополнительного столбца) небольшое и имеет большое количество пропусков, так что можно обойтись без них.

In [ ]:
df1 = df1.loc[df1['missed'].isna()]
df1.drop(['missed'], axis=1, inplace=True)
df1.shape

missed = pd.concat([df1, df2]).drop_duplicates(subset=['comments'], keep=False)
print('Nan elements missed:\n')
na_cols_freq(missed)
print('Num of latitude = 0:', missed.loc[missed['latitude'] == '0'].shape[0], '\n')
print('Num of longitude = 0:',missed.loc[missed['longitude '] == '0'].shape[0], '\n')
print('Num of duration seconds = 0:',missed.loc[missed['duration (seconds)'] == '0'].shape[0], '\n')

missed.head()

Количество данных в таблице complete на 10% больше чем в scrubbed. Так как пропусков не так много, то данная информация будет полезна и необходимо ее учитывать после предобработки. 

## Предобработка

In [ ]:
ufo_base = pd.concat([df1, df2]).drop_duplicates(subset=['comments'], keep='last')

#  изменения 24:00 на 23.59
def custom_to_datetime(date):
    if date[11:13] == '24':
        x = date[:11] + '23:59'
    elif date[10:12] == '24':
        x = date[:10] + '23:59'
    elif date[9:11] == '24':
        x = date[:9] + '23:59'
    else:
        return pd.to_datetime(date)
    return pd.to_datetime(x)

ufo_base.index = np.arange(0, len(ufo_base))
ufo_base['datetime'] = ufo_base['datetime'].apply(custom_to_datetime)  
ufo_base['date posted'] = pd.to_datetime(ufo_base['date posted'])

ufo_base['country'] = ufo_base['country'].fillna('unknown')
ufo_base['state'] = ufo_base['state'].fillna('unknown')
ufo_base['shape'] = ufo_base['shape'].fillna('unknown')
ufo_base['comments'] = ufo_base['comments'].fillna('unknown')

ufo_base['shape'] = ufo_base['shape'].astype('category')
ufo_base['state'] = ufo_base['state'].astype('category')
ufo_base['country'] = ufo_base['country'].astype('category')

ufo_base['duration (seconds)'] = pd.to_numeric(ufo_base['duration (seconds)'], errors='coerce')
ufo_base['latitude'] = pd.to_numeric(ufo_base['latitude'], errors='coerce')
ufo_base['longitude '] = pd.to_numeric(ufo_base['longitude '], errors='coerce')    

ufo_base.loc[: ,'latitude'].replace({0.0: np.nan}, inplace=True)
ufo_base.loc[: ,'longitude '].replace({0.0: np.nan}, inplace=True)
ufo_base.loc[: ,'duration (seconds)'].replace({0.0: np.nan}, inplace=True)
ufo_base['month'] = ufo_base['datetime'].dt.month
ufo_base['weekday'] = ufo_base['datetime'].dt.weekday

Количество пропусков в обработанной таблице:

In [ ]:
na_cols_freq(ufo_base)

# Визуализация:

## Плотность распределения заявок об увиденном UFO по годам

In [ ]:
plt.clf()
sns_plot = sns.distplot(ufo_base['datetime'].dt.year, bins = 54).\
set_title('Плотность распределения заявок об увиденном UFO по годам')
fig = sns_plot.get_figure()
plt.xlim(1960,2020)
plt.grid(True)
plt.xlabel("Год")
plt.ylabel("Плотность распределения")

## Количество заявок об увиденном UFO по месяцам

In [ ]:
data = ufo_base.groupby(['month']).size().reset_index(name='count')

fig, ax = plt.subplots()

ax.bar(data['month'], data['count'])
plt.xlabel('Месяц', fontsize=15)
plt.ylabel(' Количество заявок об увиденном UFO по месяцам', fontsize=11)
ax.set_xticklabels(['Январь','Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль',\
                    'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'])
plt.xticks(rotation=90)
plt.locator_params(axis='x', nbins=13)

## Количество заявок об увиденном UFO по дням недели

In [ ]:
data = ufo_base.groupby(['weekday']).size().reset_index(name='count')

fig, ax = plt.subplots()

ax.bar(data['weekday'], data['count'])
plt.xlabel('Месяц', fontsize=15)
plt.ylabel(' Количество заявок об увиденном UFO по месяцам', fontsize=11)
ax.set_xticklabels(['Понедельник', 'Понедельник', 'Вторник', 'Среда', 'Четверг',\
                    'Пятница', 'Суббота', 'Воскресенье'])
plt.xticks(rotation=90)
plt.locator_params(axis='x', nbins=8)



## Количество заявок об увиденном UFO по часам

In [ ]:
plt.clf()
sns_plot = sns.distplot(ufo_base['datetime'].dt.hour, bins = 24, kde=False).\
set_title('Количество заявок об увиденном UFO по часам')
fig = sns_plot.get_figure()
#plt.xlim(1960,2020)
plt.grid(True)

Можем заметить следующие закономерности:

1.  Чем дальше год, тем больше заявок. Это связано с тем, что население мира значительно выросло, проще стало документировать случаи (давние могли быть утеряны) и уфологи активнее начинают работать:)

2. В летнее время больше заявок. Может быть связано с тем, что люди чаще гуляют в это время по улицам.

3. Основная часть увиденных объектов замечена в вечернее и ночное время, так как легче идентифицировать объекты в небе.

## Рейтинг по частоте увиденных в небе фигур

In [ ]:


sns.set(style='darkgrid')
plt.figure(figsize=(20,10))
ax = sns.countplot(y='shape', data=ufo_base, \
                   order = ufo_base['shape'].value_counts().index).\
set_title('Рейтинг по частоте увиденных в небе фигур')
#plt.xticks(rotation=90)

plt.ylabel('Количество', fontsize=20)
plt.xlabel('Тип фигуры', fontsize=20)


Очень часто фигуры не указаны (unknown).

## Количество объявлений об увиденном UFO по штатам

In [ ]:
ax = sns.catplot(y='state',
            data=ufo_base,
            kind='count',
            height=15,
            order = ufo_base['state'].value_counts().index)

ax.fig.suptitle('Количество объявлений об увиденном UFO по штатам', x=0.5, y=1,\
                fontsize = 25)

plt.ylabel('Штат', fontsize=20)
plt.xlabel('Количество', fontsize=20)

Посмотрев штаты с наиболее частыми явлениями UFO можно заметить, что это штаты с самым высоким населением в стране.

In [ ]:
sns.set(style='darkgrid')
plt.figure(figsize=(20,10))
ax = sns.countplot(x='country', data=ufo_base,\
                  order = ufo_base['country'].value_counts().index, log=True).\
set_title('Количество объявлений об увиденном UFO по странам', fontsize=25)

plt.ylabel('Количество, log', fontsize=20)
plt.xlabel('Штат', fontsize=20)

## Продолжительность видимости UFO

In [ ]:
fig = plt.hist(x = ufo_base['duration (seconds)'], bins=np.arange(0,15000,300), log=True)

plt.title('Продолжительность видимости UFO', fontsize=20)

plt.ylabel('Количество, log', fontsize=15)
plt.xlabel('Время, с', fontsize=15)

Можно заметить, что есть скачки в гистограмме. Это связано с тем, что при записи указывали, как правило, округленное время. Например, можно заметить большие бины в значениях 3600 и 7200 (час и два часа).
В основном, увидеть UFO удавалось непродолжительное время.

In [ ]:
fig = plt.hist(x = (ufo_base['date posted'].dt.year - ufo_base['datetime'].dt.year), bins=np.arange(0,100,3), log=True)

plt.title('Разница между моментом видимости UFO и моментом размещения информации в годах', fontsize=15)

plt.ylabel('Количество, log', fontsize=15)
plt.xlabel('Разница в годах', fontsize=15)

Видно, что большая часть заявлений записана с небольшим интервалом. Это также связано с тем, что в последние года поступило большее количество объявлений.

## Наиболее часто встречаемые слова в комментариях:

In [ ]:
def text_edit(row):
    row = row.lower()
    row = re.sub('[^а-яёa-z\s]', '', row)
    row = nltk.word_tokenize(row)
    row = [stemmer.stem(x) for x in row]    
    return row
   
# приведение к нижнему регистру, удаление лишних знаков и стоп-слов.
stemmer = nltk.stem.SnowballStemmer('english')
stop_words = stopwords.words('english')

comments = ufo_base['comments'].apply(text_edit)
comments = comments.apply(lambda x: [item for item in x if item not in stop_words])
comments = list(comments.apply(lambda x: ' '.join(x)))

# векторизация, вычисление наиболее часто встречаемых слов.

count_vec = CountVectorizer(min_df = 0.01)
X = count_vec.fit_transform(comments)

vocab = count_vec.get_feature_names()
counts = X.sum(axis=0).A1
freq_distribution = Counter(dict(zip(vocab, counts)))
common = freq_distribution.most_common(50)
pd.DataFrame(common, columns= ['word', 'count'])

Можно заметить, что в комментариях чаще всего описывают:

1. Форму объекта.
2. Цвет.
3. Скорость полета.
4. В каком направлении был замечен объект.
5. Количество объектов.

## Места, где были обнаружены UFO на карте мира

In [ ]:
sample =ufo_base.loc[(~ufo_base['latitude'].isna())].sample(10000)

Map = folium.Map(zoom_start=2)
for lat, long in zip(sample['latitude'], sample['longitude ']):
    folium.Circle((lat, long),
                   radius=0.1,
                   color='red',
                   fill_color='#3186cc',
                   ).add_to(Map)
Map

Видно, что основная часть замеченных объектов располагается в англоязычных странах с высоким уровнем жизни.

In [ ]:
sample =ufo_base.loc[ (ufo_base['country'] == 'us') & (~ufo_base['latitude'].isna())].sample(5000)

Map = folium.Map(location=(41.526653, -101.103882), zoom_start=2.7)
for lat, long in zip(sample['latitude'], sample['longitude ']):
    folium.Circle((lat, long),
                   radius=0.01,
                   color='red',
                   fill_color='#3186cc',
                   ).add_to(Map)
Map

Внутри страны и на Аляске небольшая плотность из-за низкой плотности населения и малого количества городов.

## Количество заявлений по штатам США

In [ ]:
counts_of_vision = pd.DataFrame(ufo_base.groupby('state')['state'].count())
counts_of_vision['count'] = counts_of_vision['state']
x = pd.Series(counts_of_vision.index).apply(lambda x: x.upper())
counts_of_vision.index = np.arange(0, counts_of_vision.shape[0])
counts_of_vision['State'] = x
counts_of_vision.drop('state', axis=1, inplace=True)

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='количество',
    data=counts_of_vision,
    columns=['State', 'count'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.4,
    legend_name='Number of UFO Sightings'
).add_to(m)

folium.LayerControl().add_to(m)
m

Калифорния, Флорида Техас и т. д. штаты с самым большим населением, поэтому заявлений больше.

## Продолжительность наблюдения за UFO по штатам

In [ ]:
counts_of_vision = pd.DataFrame(ufo_base.groupby('state').mean())
counts_of_vision['count'] = counts_of_vision['duration (seconds)']
counts_of_vision['duration (seconds)'] = np.log(counts_of_vision['duration (seconds)'])
x = pd.Series(counts_of_vision.index).apply(lambda x: x.upper())
counts_of_vision.index = np.arange(0, counts_of_vision.shape[0])
counts_of_vision['State'] = x

bins = list(counts_of_vision['duration (seconds)'].quantile([0, 0.25, 0.5, 0.75, 1]))

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='количество',
    data=counts_of_vision,
    columns=['State', 'duration (seconds)'],
    key_on='feature.id',
    fill_color='YlOrRd',
    bins=bins,
    fill_opacity=0.7,
    line_opacity=0.4,
    legend_name='Time of UFO Sightings in log(seconds)'
).add_to(m)

folium.LayerControl().add_to(m)
m

С первого взгзяда не видно прямой зависимости. Может быть связано с авиабазами, законодательством в штате, подверженности населения уфологам и т. д.

На Аляске и Гаваях продолжительное время из-за хорошей видимости в небе.